<a href="https://www.kaggle.com/code/isithadinujaya/deep-audio-zoomoing?scriptVersionId=299191373" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Cell 1: Install dependencies (run once)
!pip install gpuRIR soundfile librosa tqdm

# Detect CUDA version and install matching cupy
import subprocess
cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
if 'release 12' in cuda_version.stdout:
    !pip install cupy-cuda12x
elif 'release 11' in cuda_version.stdout:
    !pip install cupy-cuda11x
else:
    print("CUDA version not recognised, trying default cupy")
    !pip install cupy

ERROR: Could not find a version that satisfies the requirement gpuRIR (from versions: none)
ERROR: No matching distribution found for gpuRIR


In [5]:
import os
import random
import json
import numpy as np
import soundfile as sf
import librosa
import pyroomacoustics as pra
from scipy.signal import fftconvolve
from tqdm import tqdm



In [2]:
pip install pyroomacoustics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 42.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyroomacoustics: filename=pyroomacoustics-0.9.0-cp312-cp312-linux_x86_64.whl size=47282651 sha256=4d3e08ead52e8c956dd2966d810f7784d8f64a4648dd93a31c731b63c675380a
  Stored in directory: /root/.cache/pip/wheels/ae/dc/1f/4bf34ad9ec966bb11ee31f9f92beb88772c46612865adb62f2
Successfully built pyroomacoustics
Note: you may need to restart the kernel to use updated packages.


In [7]:
sample_rate = 16000
duration_max = 8

num_mics =8

mic_radius = 0.05

mic_pos = pra.circular_2D_array(center = [0,0], M= num_mics, phi0=0, radius=mic_radius)


room_size = [5,8,3] #width, depth, height 

t_60_range =[0.3,1.3] #reverberation time

SNR_range = [10,40]
Source_dist_min = 0.5  #minimum distance from array





In [8]:
speech_root = "/kaggle/input/aishell-dataset/wav"

noise_root = "/kaggle/input/datasets/chrisfilo/demand"

output = "/kaggle/working/audi_zoom"

os.makedirs(output, exist_ok = True)

In [9]:
N_train = 5000
N_Val = 500
test = 100


In [10]:
def load_speech_files(root):
    "recursively finding all.wav files under root"
    speech_files = []

    for subdir, i, files in os.walk(root):
        for f in files:
            if f.endswith('.wav'):
                speech_files.append(os.path.join(subdir,f))

    return speech_files


def load_noise(root):

    noise_files = []

    for sundir, j, files in os.walk(root):
        for f in files:
            if f.endswith(("wav","flac")):
                noise_files.append(os.path.join(subdir,f))
    return noise_files



In [11]:
#diiferent room sizes for data set generations


def random_room_params():
    Lx = random.uniform(4,8)
    Ly = random.uniform(4,8)
    Lz = random.uniform(2.5, 4.0)

    room_sz = [Lx,Ly,Lz]
    
    margin = 0.5 #not close to room walls

    array_pos =[random.uniform(margin, Lx - margin),random.uniform(margin, Ly - margin),random.uniform(1,1.5) ]

    return room_sz, array_pos

In [12]:
def random_source_positions(num_src, room_sz, array_pos, min_dist = Source_dist_min):
    pos = []
    margin = 0.3
    for i in range(num_src):
        while True:
            p = [random.uniform(margin, room_sz[0] - margin),random.uniform(margin, room_sz[1] - margin),random.uniform(1,1.8) ]

            dist = np.linalg.norm(np.array(p) - np.array(array_pos))
        
            if dist >= min_dist :
                pos.append(p)
                break
    return np.array(poss)



In [13]:
def random_fov():

    az_width = random.uniform(30,180)*np.pi/180
    el_width = random.uniform(10,60)*np.pi/180

    az_center = random.uniform(-np.pi,np.pi)
    el_center = random.uniform(-np.pi/4,np.pi/4)

    az_min = az_center - az_width/2
    az_max = az_center + az_width/2
    el_min = el_center - el_center/2
    el_max = el_center + el_center/2

    return [az_min, az_max], [el_min, el_max]